## **Using Beam Search on our best models**

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()
from wandb.keras import WandbCallback as WC
import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 1.8MB 11.8MB/s 
     |████████████████████████████████| 174kB 36.8MB/s 
     |████████████████████████████████| 133kB 35.8MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Dense, Embedding, Input, LSTM, GRU
from keras.callbacks import Callback
from keras.models import Sequential
from tensorflow import keras
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np
import math

In [ ]:
#Data Path
train_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Train.tsv'
test_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Test.tsv'
val_path = '/content/drive/MyDrive/DL_Assignment 3/lexicons/Val.tsv'

train = pd.read_csv(train_path, delimiter='\t')
val = pd.read_csv(val_path, delimiter='\t')
test = pd.read_csv(test_path, delimiter='\t')

In [ ]:
sweep_config = {
    'method' : 'bayes',
    'metric' :
    {
        'name' : 'word_validation_accuracy',
        'goal' : 'maximize'
    },
    'parameters' :
    {
        'optimizer' : {
            'values' : ['rmsprop', 'adam']
        },
        'dropout' :
        {
            'values' : [0.3, 0.5]
        },
        'attention' :
        {
            'values' : ['no', 'yes']
        },
        'top_k' :
        {
            'values' : [2, 3]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910-assignment-3")

Create sweep with ID: 3lhapbbc
Sweep URL: https://wandb.ai/sushane/cs6910-assignment-3/sweeps/3lhapbbc


In [ ]:
max_encoder_seq_length = 0
max_decoder_seq_length = 0
num_encoder_tokens     = 0
num_decoder_tokens     = 0
input_token_index      = {}
arget_token_index      = {}
input_characters       = set()
target_characters      = set()
def Vectorize_Data(Data, isTrain):

  global input_characters, target_characters,  num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index
  input_texts       = []
  target_texts      = []
  input_characters  = set()
  target_characters = set()
  for index, line in Data.iterrows():
    input_text, target_text = line['English'], line['Hindi']
    input_text, target_text = str(input_text), str(target_text)
    target_text = " " + target_text + " "
    input_text  = " " + input_text + " "
    input_texts.append(input_text)
    target_texts.append(target_text)
    if isTrain:
      for char in input_text:
          if char not in input_characters:
              input_characters.add(char)
      for char in target_text:
          if char not in target_characters:
              target_characters.add(char)
  if isTrain:
    input_characters       = sorted(list(input_characters))
    target_characters      = sorted(list(target_characters))
    num_encoder_tokens     = len(input_characters)
    num_decoder_tokens     = len(target_characters)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])+1
    max_decoder_seq_length = max([len(txt) for txt in target_texts])+1

    input_token_index  = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

  encoder_input_data  = np.zeros((len(input_texts), max_encoder_seq_length), dtype = "float32")
  decoder_input_data  = np.zeros((len(input_texts), max_decoder_seq_length), dtype = "float32")
  decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = "float32")

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t]   =  input_token_index[char]
      encoder_input_data[i, t + 1 :] = input_token_index[" "]
      for t, char in enumerate(target_text):
          decoder_input_data[i, t]   = target_token_index[char]
          if t > 0:
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1 :] = target_token_index[" "]
      decoder_target_data[i, t:, target_token_index[" "]] = 1.0

  return input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
def beam_search_decoder(data, top_k = 3):
  
  top_k_predictions = np.zeros((len(data), top_k, data.shape[1]), dtype="int32")
  for k, predictions in enumerate(data):
    output_sequences = [([], 0)]
    for token_probs in predictions:
        new_sequences = []
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
    for i in range(top_k):
        top_k_predictions[k][i] = output_sequences[i][0]
  return top_k_predictions

In [ ]:
def RNN():

  input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data = Vectorize_Data(train, 1)
  input_texts_val, target_texts_val, encoder_input_data_val, decoder_input_data_val, decoder_target_data_val = Vectorize_Data(val, 0)
  input_texts_test, target_texts_test, encoder_input_data_test, decoder_input_data_test, decoder_target_data_test = Vectorize_Data(test, 0)

  config_defaults = {
      'optimizer'         : "adam",
      'dropout'           : 0.5,
      'attention'         : 'yes',
      'top_k'             : 2
  }

  wandb.init(project= 'cs6910-assignment-3', config = config_defaults)
  config            = wandb.config
  name              = "CELL=GRU_OP=" + str(config.optimizer) + "_BS=64" + "_D=" + str(config.dropout)
  wandb.init().name = name

  latent_dim        = 128
  embedding_size    = 256
  layer             = 3
  lr                = 1e-3
  batch_size        = 64
  optimizer         = config.optimizer
  drop              = config.dropout
  epochs            = 25
  attention         = config.attention
  top_k             = config.top_k

  #Model
  global Decoder_states, word_val_acc
  Decoder_states = []
  layer -= 1
  encoder_input     = Input(shape=(max_encoder_seq_length,))
  encoder_embedding = Embedding(input_dim = num_encoder_tokens, output_dim = embedding_size)(encoder_input)

  encoder = GRU(latent_dim, return_state = True, return_sequences = True)
  outputs, state_h = encoder(encoder_embedding)
  Decoder_states.append([state_h])
  for i in range(layer):
      encoder_gru =  GRU(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      outputs, state_h = encoder_gru(outputs)
      Decoder_states.append([state_h])

  decoder_input = Input(shape=(max_decoder_seq_length,))
  decoder_embedding = Embedding(input_dim = num_decoder_tokens, output_dim = embedding_size)(decoder_input)
  
  decoder = GRU(latent_dim, return_state = True, return_sequences = True)
  de_outputs, _ = decoder(decoder_embedding, initial_state = Decoder_states[0])
  for i in range(layer):
      decoder_gru = GRU(latent_dim, return_state = True, return_sequences = True, dropout = drop)
      de_outputs, state_h = decoder_gru(de_outputs, initial_state = Decoder_states[i+1])

  if attention == "yes":
      decoder_attention = keras.layers.AdditiveAttention(name = "decoder_attention")
      decoder_concat    = keras.layers.Concatenate(name = "decoder_concat")
      context_vec, attn_weights = decoder_attention([de_outputs, outputs], return_attention_scores = True)
      de_outputs = decoder_concat([de_outputs, context_vec])

  decoder_dense = Dense(num_decoder_tokens, activation = "softmax")
  decoder_outputs = decoder_dense(de_outputs)

  class PredictionCallback(tf.keras.callbacks.Callback):
    top_k = config.top_k
    def __init__(self, validation_data):
      self.validation_data = validation_data
    def on_epoch_end(self, epoch, logs = {}):
      predictions = self.model.predict(self.validation_data[0])
      ground_truth = self.validation_data[1]

      top_k_predictions = beam_search_decoder(predictions, self.top_k)
      count = 0
      for i in range(len(predictions)):
        
        for j in range(self.top_k):
        
          one_hot_prediction = tf.one_hot(top_k_predictions[i][j], predictions.shape[2]).numpy()
          if np.array_equal(ground_truth[i], one_hot_prediction):
            count += 1
            break

      print("Word Level Val Accuracy: " + str(count/len(predictions)))
      word_val_acc.append(count/len(predictions))

  model = keras.Model([encoder_input, decoder_input], decoder_outputs)
  word_val_acc = []
  if optimizer == "adam":
    opt = keras.optimizers.Adam(learning_rate = lr)
  if optimizer == "rmsprop":
    opt = keras.optimizers.RMSprop(learning_rate = lr)
  model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ["accuracy"])
  
  PC = PredictionCallback(validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val))
  hist = model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size = batch_size,
        epochs = epochs,
        shuffle = True,
        validation_data = ([encoder_input_data_val, decoder_input_data_val], decoder_target_data_val),
        callbacks =([PC]))
  
  
  val_acc = hist.history['val_accuracy']
  val_loss = hist.history['val_loss']
  acc = hist.history['accuracy']
  loss = hist.history['loss']

  for i in range(epochs):
    wandb.log({'word_validation_accuracy': word_val_acc[i]})
    wandb.log({'validation_accuracy': val_acc[i]})
    wandb.log({'validation_loss': val_loss[i]})
    wandb.log({'training_accuracy': acc[i]})
    wandb.log({'training_loss': loss[i]})

In [ ]:
wandb.agent('1nm6a5nb', function = RNN)

wandb: Agent Starting Run: 174oqm0d with config:
wandb: 	attention: yes
wandb: 	dropout: 0.3
wandb: 	optimizer: adam
wandb: 	top_k: 3
wandb: Currently logged in as: sushane (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
691/691 [==============================] - 87s 69ms/step - loss: 1.1591 - accuracy: 0.7354 - val_loss: 0.5459 - val_accuracy: 0.8407
Word Level Val Accuracy: 0.027076640660853604
Epoch 2/25
691/691 [==============================] - 46s 66ms/step - loss: 0.4670 - accuracy: 0.8615 - val_loss: 0.2371 - val_accuracy: 0.9284
Word Level Val Accuracy: 0.40867370353373106
Epoch 3/25
691/691 [==============================] - 45s 66ms/step - loss: 0.2450 - accuracy: 0.9240 - val_loss: 0.1930 - val_accuracy: 0.9418
Word Level Val Accuracy: 0.5204222120238642
Epoch 4/25
691/691 [==============================] - 45s 66ms/step - loss: 0.1965 - accuracy: 0.9378 - val_loss: 0.1654 - val_accuracy: 0.9480
Word Level Val Accuracy: 0.565167508031207
Epoch 5/25
691/691 [==============================] - 45s 65ms/step - loss: 0.1700 - accuracy: 0.9457 - val_loss: 0.1535 - val_accuracy: 0.9520
Word Level Val Accuracy: 0.5943093162000918
Epoch 6/25
691/691 [==============================] - 45s 

word_validation_accuracy,0.66797
_runtime,1806
_timestamp,1621406722
_step,124
validation_accuracy,0.95978
validation_loss,0.13354
training_accuracy,0.97179
training_loss,0.08472


word_validation_accuracy,▁▅▆▇▇▇▇██████████████████
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_accuracy,▁▆▇▇█▇███████████████████
validation_loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_accuracy,▁▅▇▇▇▇▇▇█████████████████
training_loss,█▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: 3ce87bkw with config:
wandb: 	attention: yes
wandb: 	dropout: 0.5
wandb: 	optimizer: adam
wandb: 	top_k: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep


...

Epoch 1/25
691/691 [==============================] - 59s 70ms/step - loss: 1.1598 - accuracy: 0.7345 - val_loss: 0.5910 - val_accuracy: 0.8290
Word Level Val Accuracy: 0.016291877007801745
Epoch 2/25
691/691 [==============================] - 46s 67ms/step - loss: 0.5254 - accuracy: 0.8450 - val_loss: 0.2654 - val_accuracy: 0.9205
Word Level Val Accuracy: 0.3483249196879302
Epoch 3/25
691/691 [==============================] - 47s 67ms/step - loss: 0.2895 - accuracy: 0.9105 - val_loss: 0.2026 - val_accuracy: 0.9380
Word Level Val Accuracy: 0.4782010096374484
Epoch 4/25
691/691 [==============================] - 46s 67ms/step - loss: 0.2307 - accuracy: 0.9278 - val_loss: 0.1816 - val_accuracy: 0.9442
Word Level Val Accuracy: 0.536714089031666
Epoch 5/25
691/691 [==============================] - 46s 67ms/step - loss: 0.2027 - accuracy: 0.9358 - val_loss: 0.1695 - val_accuracy: 0.9473
Word Level Val Accuracy: 0.5592014685635612
Epoch 6/25
691/691 [==============================] - 46s 6

word_validation_accuracy,0.65603
_runtime,1788
_timestamp,1621417351
_step,124
validation_accuracy,0.95885
validation_loss,0.13271
training_accuracy,0.9631
training_loss,0.11223


word_validation_accuracy,▁▅▆▇▇▇▇▇▇████████████████
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
validation_accuracy,▁▆▇▇▇▇███████████████████
validation_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_accuracy,▁▅▆▇▇▇▇▇█████████████████
training_loss,█▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: jep9wg9x with config:
wandb: 	attention: no
wandb: 	dropout: 0.3
wandb: 	optimizer: rmsprop
wandb: 	top_k: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Epoch 1/25
691/691 [==============================] - 60s 70ms/step - loss: 1.0751 - accuracy: 0.7405 - val_loss: 0.6597 - val_accuracy: 0.8163
Word Level Val Accuracy: 0.0032124827902707664
Epoch 2/25
691/691 [==============================] - 47s 67ms/step - loss: 0.6152 - accuracy: 0.8252 - val_loss: 0.3897 - val_accuracy: 0.8832
Word Level Val Accuracy: 0.12574575493345572
Epoch 3/25
691/691 [==============================] - 47s 67ms/step - loss: 0.3860 - accuracy: 0.8834 - val_loss: 0.2607 - val_accuracy: 0.9215
Word Level Val Accuracy: 0.3533731069297843
Epoch 4/25
691/691 [==============================] - 47s 67ms/step - loss: 0.2794 - accuracy: 0.9138 - val_loss: 0.2169 - val_accuracy: 0.9343
Word Level Val Accuracy: 0.44125745754933454
Epoch 5/25
691/691 [==============================] - 47s 68ms/step - loss: 0.2268 - accuracy: 0.9294 - val_loss: 0.1903 - val_accuracy: 0.9422
Word Level Val Accuracy: 0.5094079853143644
Epoch 6/25
691/691 [==============================] - 4